# USGS E. coli Prediction Models Recreation

This notebook contains recreations of multiple linear regression models utilised by the USGS to predict E. coli concentration sites on two different lakes

**Models:**
1. **Huntington Beach** (Pennsylvania) - Predicts LOG10[E. coli CFU/100mL]
2. **Beach6** (Ohio) - Predicts E. coli LOG10 concentrations

requires hw1.py to be in same directory as the notebook to run it

## setup and import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from model import HuntingtonEcoliModel, Beach6EcoliModel, print_model_summary

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

pd.set_option('display.max_columns',None)
pd.set_option('display.precision',4)

print("imports successful")
print(f"working directory: {Path.cwd()}")

## Huntington Beach Model

### load & explore data

In [ ]:
hunt_model = HuntingtonEcoliModel()

hunt_df = hunt_model.load_data('models/Huntington_MAS_pkg/Huntington_2019_calibration_data.csv')

print(f"Huntington Beach Dataset")
print(f"Shape: {hunt_df.shape}")
print(f"Date range: {hunt_df['Date'].min()} to {hunt_df['Date'].max()}")
print(f"\nFirst 10 rows:")
hunt_df.head(10)

In [ ]:
print("Summary Statistics - Huntington Beach")
hunt_df.describe()

### check for missing values

In [ ]:
print("Missing Values - Huntington Beach")
missing = hunt_df.isnull().sum()
missing_pct = (missing / len(hunt_df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("\nNo missing values found")

### raw data visualised:

In [ ]:
# visualisations for Huntington data
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Huntington Beach - Raw Variable Distributions', fontsize=16, fontweight='bold')

# E. coli distribution
axes[0, 0].hist(hunt_df['EcoliAve_CFU'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('E. coli (CFU/100mL)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('E. coli Distribution')
axes[0, 0].axvline(235, color='red', linestyle='--', label='EPA Standard (235)')
axes[0, 0].legend()

# temperature
axes[0, 1].hist(hunt_df['Lake_Temp_C'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Lake Temperature (°C)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Lake Temperature Distribution')

# turbidity
axes[0, 2].hist(hunt_df['Lake_Turb_NTRU'], bins=50, edgecolor='black', alpha=0.7, color='brown')
axes[0, 2].set_xlabel('Turbidity (NTU)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('Turbidity Distribution')

# wave heights
axes[1, 0].hist(hunt_df['WaveHt_Ft'], bins=30, edgecolor='black', alpha=0.7, color='blue')
axes[1, 0].set_xlabel('Wave Height (ft)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Wave Height Distribution')

# lake level changes
axes[1, 1].hist(hunt_df['LL_PreDay'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].set_xlabel('Lake Level Change (ft)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Lake Level Change Distribution')

# rainfall
axes[1, 2].hist(hunt_df['AirportRain48W_in'], bins=40, edgecolor='black', alpha=0.7, color='purple')
axes[1, 2].set_xlabel('48hr Rainfall (in)')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].set_title('48hr Rainfall Distribution')

plt.tight_layout()
plt.savefig('paper/figures/huntington_distributions.png', dpi=300, bbox_inches='tight')
print("Saved: paper/figures/huntington_distributions.png")
plt.show()

### establish features & train model:

In [ ]:
hunt_features = hunt_model.create_features(hunt_df)

print("Transformed Features for Huntington Beach")
print(f"Shape: {hunt_features.shape}")
print(f"\nNew feature columns:")
print(hunt_features[['LOG10_EcoliAve_CFU', 'LOG10_Lake_Turb_NTRU', 
                            'SQRT_WaveHt_Ft', 'SQRT_AirportRain48W_in']].head())

### fitting model:

In [ ]:
hunt_model.fit(hunt_features)

print("Huntington Beach model fitted successfully")

### results & comparison to USGS model

In [ ]:
hunt_metrics = hunt_model.evaluate(hunt_features)

# model summary
print_model_summary(hunt_model, "Huntington Beach (Pennsylvania)",  hunt_metrics, hunt_features)

### comparing coefficients with USGS' model

In [ ]:
print("\nCoefficient Comparison: Our Model vs. USGS Model")
hunt_comparison = hunt_model.compare_with_usgs()
hunt_comparison

In [ ]:
print("\nPerformance Metrics Comparison: Our Model vs. USGS Model")
usgs_metrics = HuntingtonEcoliModel.USGS_METRICS
comp_data = []

for metric_name in ['r_squared', 'adj_r_squared', 'rmse', 'sensitivity', 'specificity', 'accuracy']:
    our_val = hunt_metrics.get(metric_name, np.nan)
    usgs_val = usgs_metrics.get(metric_name, np.nan)
    diff = our_val - usgs_val
    
    comp_data.append({
        'Metric': metric_name,
        'Our Model': our_val,
        'USGS Model': usgs_val,
        'Difference': diff
    })

metrics_comparison_df = pd.DataFrame(comp_data)
metrics_comparison_df

### model perf visualised:

In [ ]:
# model performance predictions visualised:
hunt_preds = hunt_model.predict(hunt_features)
hunt_actual = hunt_features['LOG10_EcoliAve_CFU']
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Huntington Beach Model Performance', fontsize=16, fontweight='bold')

# predicted vs actual
axes[0].scatter(hunt_actual, hunt_preds, alpha=0.5, edgecolor='black')
axes[0].plot([hunt_actual.min(), hunt_actual.max()], 
             [hunt_actual.min(), hunt_actual.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual LOG10[E. coli]')
axes[0].set_ylabel('Predicted LOG10[E. coli]')
axes[0].set_title('Predicted vs Actual')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# r-squared plot
r2 = hunt_metrics['r_squared']
axes[0].text(0.05, 0.95, f'R² = {r2:.4f}', 
            transform=axes[0].transAxes, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# residuals plot
residuals = hunt_actual - hunt_preds
axes[1].scatter(hunt_preds, residuals, alpha=0.5, edgecolor='black')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted LOG10[E. coli]')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')
axes[1].grid(True, alpha=0.3)

# residuals distribution
axes[2].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[2].axvline(x=0, color='r', linestyle='--', lw=2)
axes[2].set_xlabel('Residuals')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Residual Distribution')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('paper/figures/huntington_performance.png', dpi=300, bbox_inches='tight')
print("Saved: paper/figures/huntington_performance.png")
plt.show()

In [ ]:
# feature importance
coefs = hunt_model.get_coefficients()
features = hunt_model.feature_names
coef_values = [abs(coefs[feat]) for feat in features]

plt.figure(figsize=(10, 6))
plt.barh(features, coef_values, edgecolor='black', alpha=0.7)
plt.xlabel('Absolute Coefficient Value')
plt.title('Huntington Beach: Feature Importance (Coefficient Magnitudes)', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('paper/figures/huntington_feature_importance.png', dpi=300, bbox_inches='tight')
print("Saved: paper/figures/huntington_feature_importance.png")
plt.show()

## Beach6 Model (Ohio)

### loading data

In [ ]:
beach6_model = Beach6EcoliModel()
beach6_df = beach6_model.load_data('models/Beach6_MAS_pkg/Beach6_2019_calibration_data.csv')

print(f"Beach6 Dataset")
print(f"Shape: {beach6_df.shape}")
print(f"Date range: {beach6_df['DATE'].min()} to {beach6_df['DATE'].max()}")
print(f"\nFirst 10 rows:")
beach6_df.head(10)

In [ ]:
print("Summary Statistics - Beach6")
beach6_df.describe()

### check for missing values in dataset:

In [ ]:
print("Missing Values: Beach6")
missing = beach6_df.isnull().sum()
missing_pct = (missing / len(beach6_df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("\nNo missing values found")

### visualise the raw data:

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 10))
fig.suptitle('Beach6 - Raw Variable Distributions', fontsize=16, fontweight='bold')

# E. coli (already in LOG10)
axes[0, 0].hist(beach6_df['ECOLI_LOG10'], bins=40, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('E. coli (LOG10)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('E. coli LOG10 Distribution')
axes[0, 0].axvline(np.log10(235), color='red', linestyle='--', label='EPA Standard')
axes[0, 0].legend()

# turbidity
axes[0, 1].hist(beach6_df['TURB_NTRU'], bins=50, edgecolor='black', alpha=0.7, color='brown')
axes[0, 1].set_xlabel('Turbidity (NTU)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Turbidity Distribution')

# rel. humidity
axes[0, 2].hist(beach6_df['RHUM_PCT'], bins=30, edgecolor='black', alpha=0.7, color='cyan')
axes[0, 2].set_xlabel('Relative Humidity (%)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('Humidity Distribution')

# water temp
axes[0, 3].hist(beach6_df['WTEMP_CEL'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0, 3].set_xlabel('Water Temperature (in celsius)')
axes[0, 3].set_ylabel('Frequency')
axes[0, 3].set_title('Water Temperature Distribution')

# birds
axes[1, 0].hist(beach6_df['BIRDS_NO'], bins=40, edgecolor='black', alpha=0.7, color='yellow')
axes[1, 0].set_xlabel('Number of Birds')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Bird Count Distribution')

# lake level changes
axes[1, 1].hist(beach6_df['CHANGELL_FT'], bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].set_xlabel('Lake Level Change (ft)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Lake Level Change Distribution')

# wind speed
axes[1, 2].hist(beach6_df['AirportWindSpInst_mph'], bins=30, edgecolor='black', alpha=0.7, color='blue')
axes[1, 2].set_xlabel('Wind Speed (mph)')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].set_title('Wind Speed Distribution')

# rainfall
axes[1, 3].hist(beach6_df['AirportRain48W_in'], bins=40, edgecolor='black', alpha=0.7, color='purple')
axes[1, 3].set_xlabel('48hr Rainfall (inches)')
axes[1, 3].set_ylabel('Frequency')
axes[1, 3].set_title('48hr Rainfall Distribution')

plt.tight_layout()
plt.savefig('paper/figures/beach6_distributions.png', dpi=300, bbox_inches='tight')
print("Saved: paper/figures/beach6_distributions.png")
plt.show()

### creating features + training model

In [ ]:
beach6_features = beach6_model.create_features(beach6_df)

print("Transformed Features: Beach6")
print(f"Shape: {beach6_features.shape}")
print(f"\nNew feature columns:")
print(beach6_features[['ECOLI_LOG10', 'LOG10_TURB_NTRU', 'SQRT_AirportRain48W_in']].head())

In [ ]:
beach6_model.fit(beach6_features)

print("Beach6 model fitted successfully")

### compare model results to USGS model results

In [ ]:
beach6_metrics = beach6_model.evaluate(beach6_features)
print_model_summary(beach6_model, "Beach6 (Ohio)", 
                   beach6_metrics, beach6_features)

### Compare coefficients to the USGS model

In [ ]:
print("\nCoefficient Comparison: Our Model vs. USGS Model")
beach6_comparison = beach6_model.compare_with_usgs()
beach6_comparison

In [ ]:
# comparing metrics with USGS
print("\nPerformance Metrics Comparison: Our Model vs. USGS Model")

usgs_metrics = Beach6EcoliModel.USGS_METRICS
comp_data = []

for metric_name in ['r_squared', 'adj_r_squared', 'rmse', 'sensitivity', 'specificity', 'accuracy']:
    our_val = beach6_metrics.get(metric_name, np.nan)
    usgs_val = usgs_metrics.get(metric_name, np.nan)
    diff = our_val - usgs_val
    
    comp_data.append({
        'Metric': metric_name,
        'Our Model': our_val,
        'USGS Model': usgs_val,
        'Difference': diff
    })

metrics_comp = pd.DataFrame(comp_data)
metrics_comp

### Visualise our model performance:

In [ ]:
# predictions
beach6_preds = beach6_model.predict(beach6_features)
beach6_actual = beach6_features['ECOLI_LOG10']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Beach6 Model Performance', fontsize=16, fontweight='bold')

# predicted vs actual
axes[0].scatter(beach6_actual, beach6_preds, alpha=0.5, edgecolor='black')
axes[0].plot([beach6_actual.min(), beach6_actual.max()], 
             [beach6_actual.min(), beach6_actual.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual LOG10[E. coli]')
axes[0].set_ylabel('Predicted LOG10[E. coli]')
axes[0].set_title('Predicted vs Actual')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# r-squared plot
r2 = beach6_metrics['r_squared']
axes[0].text(0.05, 0.95, f'R² = {r2:.4f}', 
            transform=axes[0].transAxes, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# residuals plot
residuals = beach6_actual - beach6_preds
axes[1].scatter(beach6_preds, residuals, alpha=0.5, edgecolor='black')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted LOG10[E. coli]')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residual Plot')
axes[1].grid(True, alpha=0.3)

# residuals distribution
axes[2].hist(residuals, bins=40, edgecolor='black', alpha=0.7)
axes[2].axvline(x=0, color='r', linestyle='--', lw=2)
axes[2].set_xlabel('Residuals')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Residual Distribution')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('paper/figures/beach6_performance.png', dpi=300, bbox_inches='tight')
print("Saved: paper/figures/beach6_performance.png")
plt.show()

### display feature importance:

In [ ]:
coefs = beach6_model.get_coefficients()
features = beach6_model.feature_names
coef_values = [abs(coefs[feat]) for feat in features]

plt.figure(figsize=(10, 6))
plt.barh(features, coef_values, edgecolor='black', alpha=0.7)
plt.xlabel('Absolute Coefficient Value')
plt.title('Beach6: Feature Importance (Coefficient Magnitudes)', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('paper/figures/beach6_feature_importance.png', dpi=300, bbox_inches='tight')
print("Saved: paper/figures/beach6_feature_importance.png")
plt.show()

### model comparison summary:

In [ ]:
print("FINAL MODEL COMPARISON SUMMARY")

comparison_data = {
    'Metric': ['N Observations', 'R²', 'Adjusted R²', 'RMSE', 'Sensitivity', 'Specificity', 'Accuracy'],
    'Huntington (Our)': [
        hunt_metrics['n_observations'],
        hunt_metrics['r_squared'],
        hunt_metrics['adj_r_squared'],
        hunt_metrics['rmse'],
        hunt_metrics['sensitivity'],
        hunt_metrics['specificity'],
        hunt_metrics['accuracy']
    ],
    'Huntington (USGS)': [
        1011,
        HuntingtonEcoliModel.USGS_METRICS['r_squared'],
        HuntingtonEcoliModel.USGS_METRICS['adj_r_squared'],
        HuntingtonEcoliModel.USGS_METRICS['rmse'],
        HuntingtonEcoliModel.USGS_METRICS['sensitivity'],
        HuntingtonEcoliModel.USGS_METRICS['specificity'],
        HuntingtonEcoliModel.USGS_METRICS['accuracy']
    ],
    'Beach6 (Our)': [
        beach6_metrics['n_observations'],
        beach6_metrics['r_squared'],
        beach6_metrics['adj_r_squared'],
        beach6_metrics['rmse'],
        beach6_metrics['sensitivity'],
        beach6_metrics['specificity'],
        beach6_metrics['accuracy']
    ],
    'Beach6 (USGS)': [
        463,
        Beach6EcoliModel.USGS_METRICS['r_squared'],
        Beach6EcoliModel.USGS_METRICS['adj_r_squared'],
        Beach6EcoliModel.USGS_METRICS['rmse'],
        Beach6EcoliModel.USGS_METRICS['sensitivity'],
        Beach6EcoliModel.USGS_METRICS['specificity'],
        Beach6EcoliModel.USGS_METRICS['accuracy']
    ]
}

final_comp = pd.DataFrame(comparison_data)
final_comp

### comparison visualised:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# r-sq comparison
models = ['Huntington\n(Our)', 'Huntington\n(USGS)', 'Beach6\n(Our)', 'Beach6\n(USGS)']
r2_values = [
    hunt_metrics['r_squared'],
    HuntingtonEcoliModel.USGS_METRICS['r_squared'],
    beach6_metrics['r_squared'],
    Beach6EcoliModel.USGS_METRICS['r_squared']
]
colors = ['#3498db', '#5dade2', '#e74c3c', '#ec7063']
axes[0].bar(models, r2_values, color=colors, edgecolor='black', alpha=0.7)
axes[0].set_ylabel('R² Value')
axes[0].set_title('Model R² Comparison', fontweight='bold')
axes[0].set_ylim([0, 0.7])
axes[0].grid(True, alpha=0.3, axis='y')

# accuracy comparison
accuracy_values = [
    hunt_metrics['accuracy'],
    HuntingtonEcoliModel.USGS_METRICS['accuracy'],
    beach6_metrics['accuracy'],
    Beach6EcoliModel.USGS_METRICS['accuracy']
]

axes[1].bar(models, accuracy_values, color=colors, edgecolor='black', alpha=0.7)
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Model Accuracy Comparison', fontweight='bold')
axes[1].set_ylim([0, 1.0])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('paper/figures/model_comparison.png', dpi=300, bbox_inches='tight')
print("Saved: paper/figures/model_comparison.png")
plt.show()

## exporting results:

In [ ]:
# Save coefficient comparisons
hunt_comparison.to_csv('paper/hunt_coeff.csv', index=False)
beach6_comparison.to_csv('paper/beach6_coeff_comp.csv', index=False)
print("Coefficient comparisons saved to paper/..")


In [ ]:
# Save predictions

hunt_results = hunt_features.copy()
hunt_results['Predicted_LOG10'] = hunt_preds
hunt_results['Predicted_CFU'] = hunt_model.predict_concentration(hunt_features)
hunt_results.to_csv('paper/hunt_preds.csv', index=False)

beach6_results = beach6_features.copy()
beach6_results['Predicted_LOG10'] = beach6_preds
beach6_results['Predicted_CFU'] = beach6_model.predict_concentration(beach6_features)
beach6_results.to_csv('paper/beach6_preds.csv', index=False)
print("Predictions saved to paper/..")
